In [359]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import folium # map rendering library

import pickle

print('Libraries imported.')

Libraries imported.


#### Define Foursquare Credentials and Version

In [360]:
CLIENT_ID = '3XK1UBWEBDUCTRUVD5G51DEIH2B2KJLULMKAD4XV4UG01RC5' # your Foursquare ID
CLIENT_SECRET = '0R4324WC0IZQOMWTMDVXP4JQRONDDNEWJIQ0P421R3CIYJDA' # your Foursquare Secret
VERSION = '20190308' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3XK1UBWEBDUCTRUVD5G51DEIH2B2KJLULMKAD4XV4UG01RC5
CLIENT_SECRET:0R4324WC0IZQOMWTMDVXP4JQRONDDNEWJIQ0P421R3CIYJDA


In [502]:
# san francisco postal codes
postal_codes = pd.read_pickle("data/sf_geospatial_data_zip.pkl")

# san francisco nieghbourhoods
neighbourhoods = pd.read_pickle("data/sf_geospatial_data.pkl")

In [503]:
neighbourhoods.count()

Neighbourhood    116
latitude         116
longitude        116
dtype: int64

In [504]:
neighbourhoods.head()

,Neighbourhood,latitude,longitude
0,Seacliff,37.785941,-122.490700
1,Lake Street,37.786197,-122.475521
2,Presidio National Park,37.807044,-122.472101
3,Presidio Terrace,37.788263,-122.460805
4,Inner Richmond,37.779870,-122.464746


In [505]:
postal_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 3 columns):
postal_code    27 non-null object
latitude       27 non-null float64
longitude      27 non-null float64
dtypes: float64(2), object(1)
memory usage: 728.0+ bytes


In [506]:
postal_codes.head()

,postal_code,latitude,longitude
0,94124,37.730417,-122.384425
1,94127,37.734646,-122.463708
2,94131,37.740104,-122.438231
3,94133,37.805989,-122.409915
4,94132,37.718140,-122.486349


In [27]:
def get_categories():
    url = 'https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION)
    results = requests.get(url).json()
    return results

In [32]:
result = get_categories()


In [94]:
def build_categories(categories, parent_id):
    df = pd.DataFrame()
    if(isinstance(categories, list)):
        for category in categories:
            icon = category['icon']['prefix'] + '32' + category['icon']['suffix']
            name = category['name']
            category_id = category['id']
            df.set_value(category_id, 'name', name)
            df.set_value(category_id, 'icon', icon)
            df.set_value(category_id, 'parent_id', parent_id)
            for sub_category in category['categories']:
                df_sub_category = build_categories(sub_category, category_id)
                df = df.append(df_sub_category)
    elif(isinstance(categories, dict)):
        icon = categories['icon']['prefix'] + '32' + categories['icon']['suffix']
        name = categories['name']
        category_id = categories['id']
        df.set_value(category_id, 'name', name)
        df.set_value(category_id, 'icon', icon)
        df.set_value(category_id, 'parent_id', parent_id)
        for sub_category in categories['categories']:
            df_sub_category = build_categories(sub_category, category_id)
            df = df.append(df_sub_category)
    return df

In [95]:
categories = build_categories(result['response']['categories'], None)

In [350]:
categories.head()

,name,icon,parent_id
4d4b7104d754a06370d81259,Arts & Entertainment,https://ss3.4sqi.net/img/categories_v2/arts_en...,NaN
56aa371be4b08b9a8d5734db,Amphitheater,https://ss3.4sqi.net/img/categories_v2/arts_en...,4d4b7104d754a06370d81259
4fceea171983d5d06c3e9823,Aquarium,https://ss3.4sqi.net/img/categories_v2/arts_en...,4d4b7104d754a06370d81259
4bf58dd8d48988d1e1931735,Arcade,https://ss3.4sqi.net/img/categories_v2/arts_en...,4d4b7104d754a06370d81259
4bf58dd8d48988d1e2931735,Art Gallery,https://ss3.4sqi.net/img/categories_v2/arts_en...,4d4b7104d754a06370d81259


In [351]:
categories.to_pickle("data/categories.pkl")

In [400]:
def explore(latitude, longitude, radius):
    LIMIT = 50
    url_string = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'
    url = url_string.format(CLIENT_ID, CLIENT_SECRET, VERSION, latitude, longitude, radius, LIMIT)
    results = requests.get(url).json()
    total_results = results['response']['totalResults']
    items = results['response']['groups'][0]['items']
    
    i = 0
    while i < total_results/LIMIT:
        url_string = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&offset={}'
        url = url_string.format(CLIENT_ID, CLIENT_SECRET, VERSION, latitude, longitude, radius, LIMIT, (i+1)*50)
        results = requests.get(url).json()
        items.extend(results['response']['groups'][0]['items'])
        i = i + 1
    
    return items


def get_venues(neighbourhoods, column, radius = 5000):
    df = pd.DataFrame()
    for index, row in neighbourhoods.iterrows():
        postal_code = row[column]
        lat = row['latitude']
        lng = row['longitude']
        print("Getting venues for {}".format(postal_code))
        venues = explore(lat, lng, radius)
        for venue in venues:
            df = df.append(parse_venue(venue['venue'], postal_code))
    return df

def parse_venue(venue, postal_code = None):
    df = pd.DataFrame()
    venue_id = venue['id']
    location = venue['location']
    df.set_value(venue_id, 'name', venue['name'])
    df.set_value(venue_id, 'lat', location['lat'])
    df.set_value(venue_id, 'lng', location['lng'])

    if 'postalCode' in location:
        df.set_value(venue_id, 'postal_code', location['postalCode'])
        
    if 'city' in location:
        df.set_value(venue_id, 'city', location['city'])
        
    if 'state' in location:
        df.set_value(venue_id, 'state', location['state'])
        
    if 'country' in location:
        df.set_value(venue_id, 'country', location['country'])
        
    if 'crossStreet' in location:
        df.set_value(venue_id, 'cross_street', location['crossStreet'])
        
    if 'address' in location:
        df.set_value(venue_id, 'address', location['address'])

    for category in venue['categories']:
        if category['primary']:
            df.set_value(venue_id, 'category', category['id'])
            break;
    return df

In [386]:
#  get venues by postal code
venues_by_postal_codes = get_venues(postal_codes, column = 'postal_code')

Getting venues for 94124
Getting venues for 94127
Getting venues for 94131
Getting venues for 94133
Getting venues for 94132
Getting venues for 94133
Getting venues for 94134
Getting venues for 94102
Getting venues for 94158
Getting venues for 94158
Getting venues for 94158
Getting venues for 94103
Getting venues for 94104
Getting venues for 94105
Getting venues for 94107
Getting venues for 94108
Getting venues for 94109
Getting venues for 94110
Getting venues for 94111
Getting venues for 94112
Getting venues for 94114
Getting venues for 94115
Getting venues for 94116
Getting venues for 94117
Getting venues for 94118
Getting venues for 94121
Getting venues for 94121
Getting venues for 94121
Getting venues for 94122
Getting venues for 94123
Getting venues for 94129
Getting venues for 94130


In [387]:
venues_by_postal_codes.shape

(7585, 10)

In [388]:
venues_by_postal_codes.head()

,address,category,city,country,cross_street,lat,lng,name,postal_code,state
4f3709abe4b0aa62a1d35520,748 Innes Ave,4bf58dd8d48988d1ed941735,San Francisco,United States,NaN,37.730525,-122.372998,Archimedes Banya,94124,CA
49d3e08bf964a5201f5c1fe3,1634 Jerrold Ave,4bf58dd8d48988d11b951735,San Francisco,United States,NaN,37.739589,-122.390171,Flora Grubb Gardens,94124,CA
53058fdb498e2837542f7f8f,1439 Egbert Ave Unit c,4e0e22f5a56208c4ea9a85a0,San Francisco,United States,3rd Street,37.723004,-122.393615,Seven Stills Brewery & Distillery,94124,CA
555e58ad498e9137bc49f191,900 Marin St,4bf58dd8d48988d1e5931735,San Francisco,United States,btwn Illinois & Michigan St,37.749404,-122.386016,The Midway SF,94124,CA
56eb4e0a498e5769545c1843,1525 Cortland Ave,50327c8591d4c4b30a586d5d,San Francisco,United States,NaN,37.739798,-122.408986,Barebottle Brewing Company,NaN,CA


In [406]:
venues_by_postal_codes.to_pickle("data/venues_by_postal_code.pkl")

In [402]:
# get venues by neighbourhoods
venues_by_neighbourhoods = get_venues(neighbourhoods, column='Neighbourhood', radius=500)

Getting venues for Seacliff
Getting venues for Lake Street
Getting venues for Presidio National Park
Getting venues for Presidio Terrace
Getting venues for Inner Richmond
Getting venues for Sutro Heights
Getting venues for Lincoln Park / Ft. Miley
Getting venues for Outer Richmond
Getting venues for Golden Gate Park
Getting venues for Presidio Heights
Getting venues for Laurel Heights / Jordan Park
Getting venues for Lone Mountain
Getting venues for Anza Vista
Getting venues for Cow Hollow
Getting venues for Union Street
Getting venues for Nob Hill
Getting venues for Marina
Getting venues for Telegraph Hill
Getting venues for Downtown / Union Square
Getting venues for Tenderloin
Getting venues for Civic Center
Getting venues for Hayes Valley
Getting venues for Alamo Square
Getting venues for Panhandle
Getting venues for Haight Ashbury
Getting venues for Lower Haight
Getting venues for Mint Hill
Getting venues for Duboce Triangle
Getting venues for Cole Valley
Getting venues for Rincon 

In [403]:
venues_by_neighbourhoods.shape

(5464, 10)

In [405]:
venues_by_neighbourhoods.head()

,address,category,city,country,cross_street,lat,lng,name,postal_code,state
408aff80f964a520c4f21ee3,Sea Cliff Ave,4bf58dd8d48988d1e2941735,San Francisco,United States,at El Camino Del Mar,37.788090,-122.491186,China Beach,94121,CA
4b11bda6f964a520918323e3,Lands End,4bf58dd8d48988d165941735,San Francisco,United States,NaN,37.786714,-122.494810,Eagles Point,NaN,CA
4be606ba910020a1bfe0d314,NaN,4f2a25ac4b909258e854f55f,San Francisco,United States,NaN,37.788259,-122.486401,Seacliff,94121,CA
4bddd43b0ee3a59333d12eb0,California St,4bf58dd8d48988d159941735,San Francisco,United States,btwn 32nd & 33rd Ave,37.783496,-122.493597,Lincoln Park Stairs,94121,CA
58a14c5d16f000694e6b9ee9,2960 Clement St,4bf58dd8d48988d16d941735,San Francisco,United States,31st Ave,37.781774,-122.491252,Saltroot Café,94121,CA


In [407]:
venues_by_neighbourhoods.to_pickle("data/venues_by_neighbourhoods.pkl")

In [417]:
venues_by_postal_codes.index.name = 'venue_id'
venues_by_neighbourhoods.index.name = 'venue_id'

In [453]:
v1 = venues_by_postal_codes.reset_index(drop=False)
v2 = venues_by_neighbourhoods.reset_index(drop=False)
v = v1.append(v2, ignore_index=True)

In [454]:
v.shape

(13049, 11)

In [455]:
v.head()

,venue_id,address,category,city,country,cross_street,lat,lng,name,postal_code,state
0,4f3709abe4b0aa62a1d35520,748 Innes Ave,4bf58dd8d48988d1ed941735,San Francisco,United States,NaN,37.730525,-122.372998,Archimedes Banya,94124,CA
1,49d3e08bf964a5201f5c1fe3,1634 Jerrold Ave,4bf58dd8d48988d11b951735,San Francisco,United States,NaN,37.739589,-122.390171,Flora Grubb Gardens,94124,CA
2,53058fdb498e2837542f7f8f,1439 Egbert Ave Unit c,4e0e22f5a56208c4ea9a85a0,San Francisco,United States,3rd Street,37.723004,-122.393615,Seven Stills Brewery & Distillery,94124,CA
3,555e58ad498e9137bc49f191,900 Marin St,4bf58dd8d48988d1e5931735,San Francisco,United States,btwn Illinois & Michigan St,37.749404,-122.386016,The Midway SF,94124,CA
4,56eb4e0a498e5769545c1843,1525 Cortland Ave,50327c8591d4c4b30a586d5d,San Francisco,United States,NaN,37.739798,-122.408986,Barebottle Brewing Company,NaN,CA


In [456]:
v.drop_duplicates(inplace=True)

In [457]:
v.shape

(4287, 11)

In [458]:
v[v['city'] != 'San Francisco']

,venue_id,address,category,city,country,cross_street,lat,lng,name,postal_code,state
77,4ed28b626da162f1c10ddf10,"Cow Palace Exhibition Halls, 2600 Geneva Ave",4bf58dd8d48988d137941735,Daly City,United States,NaN,37.705807,-122.417424,The Great Dickens Christmas Fair,94014,CA
163,4ea98253e3005b03fea315c4,Lagoon Rd,4bf58dd8d48988d161941735,Brisbane,United States,at Sierra Point Pkwy,37.693465,-122.391752,Lagoon,NaN,CA
164,50cbd454e4b0bd9096914101,1304 Howard Ave,4bf58dd8d48988d1f8941735,Burlingame,United States,NaN,37.757480,-122.347437,Floorcraft,94010,CA
165,4c48a19731e41b8df2c05035,150 N Hill Dr Ste 11A,4bf58dd8d48988d16d941735,Brisbane,United States,NaN,37.691805,-122.405739,Le Gourmet,94005,CA
238,4d4134d81bd2a143d0fbfa7c,NaN,4bf58dd8d48988d1e2941735,NaN,United States,NaN,37.741547,-122.506839,Edge Of The Western World,NaN,California
348,4adba6f4f964a520c72921e3,66 School St,4eb1bd1c3b7b55596b4a748f,Daly City,United States,NaN,37.693454,-122.465612,Fil-Am BBQ,94014,CA
382,4cb100f3ef1b370480f93f00,2152 Skyline Blvd,4eb1baf03b7b2c5b1d4306ca,Daly City,United States,NaN,37.700338,-122.496388,Mar Vista Stables,94015,CA
905,56a2acbd498ee1a54d04a90b,11 Glenwood Ave,4bf58dd8d48988d110941735,Daly City,United States,at Lake Merced Blvd,37.701560,-122.485494,Original Joe's Westlake,94015,CA
908,4a5798e3f964a52071b61fe3,417 Westlake Center,4bf58dd8d48988d118951735,Daly City,United States,Westlake Shopping Center,37.700376,-122.482614,Trader Joe's,94015,CA
914,4c1923b16a21c9b66e19ca97,Thornton Beach Rd,4bf58dd8d48988d1e2941735,Daly City,United States,at Skyline Blvd,37.698195,-122.495731,Thornton State Beach,94015,CA


In [459]:
venues = v
venues.set_index('venue_id', inplace=True)
venues.drop(venues[venues['city'] != 'San Francisco'].index, inplace=True)
venues.to_pickle("data/venues.pkl")

In [460]:
venues.shape

(4167, 10)

In [484]:
def next_venues(venue_id):
    url = 'https://api.foursquare.com/v2/venues/{}/nextvenues?&client_id={}&client_secret={}&v={}'.format(
    venue_id,
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION)
    results = requests.get(url).json()
    if 'nextVenues' in results['response']:
        if 'items' in results['response']['nextVenues']:
            return results['response']['nextVenues']['items']
    print("Error while getting next venues for {}".format(venue_id))
    return []


def get_next_venues(venues):
    df = pd.DataFrame()
    prev_run_venue_ids = set()
    same_run_count = 0
    while len(venues['gotNextVenues'] == False) != 0:
        venue_ids = venues[venues['gotNextVenues'] == False].index
        print("No. of venues: {}".format(len(venue_ids)))
        i = 0
        
        if len(set(venue_ids) - prev_run_venue_ids) == 0:
            same_run_count = same_run_count + 1
            print("Venues not processed")
            
            if same_run_count % 5 == 0:
                break;
        
        for venue_id in venue_ids:
            print("Getting next venues for {}: {}".format(i, venue_id))
            next_venues_list = next_venues(venue_id)

            venues_list = []
            venues_list.append([(
                venue_id, 
                v['id']) for v in next_venues_list])

            next_venues_df = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
            df = df.append(next_venues_df)

            venues.set_value(venue_id, 'gotNextVenues', True)

            # finding the new venues
            for v in next_venues_list:
                if v['id'] not in venues.index:
                    venue = parse_venue(v)
                    # for the next iteration get the next venues only if 
                    # the new venue is in San Francisco
                    location = v['location']
                    if 'city' in location:
                        if v['location']['city'] == 'San Francisco':
                            venue['gotNextVenues'] = False
                        else:
                            venue['gotNextVenues'] = True
                    else:
                        venue['gotNextVenues'] = True
                    venues = venues.append(venue)
            i = i + 1
            if i % 10 == 0:
                df.to_pickle("data/next_venues.pkl")
                venues.to_pickle("data/venues_processed.pkl")
                
        prev_run_venue_ids = set(venue_ids)
                
    print("Saving.............................................")
            
    df.columns = ["venue_id", "next_venue_id"]
    df.reset_index(inplace=True, drop=True)
    
    df.to_pickle("data/next_venues.pkl")
    venues.to_pickle("data/venues_processed.pkl")
    
    return df, venues

In [485]:
venues['gotNextVenues'] = False
next_v, new_venues = get_next_venues(venues)

No. of venues: 4167
Getting next venues for 0: 4f3709abe4b0aa62a1d35520
Getting next venues for 1: 49d3e08bf964a5201f5c1fe3
Getting next venues for 2: 53058fdb498e2837542f7f8f
Getting next venues for 3: 555e58ad498e9137bc49f191
Getting next venues for 4: 56eb4e0a498e5769545c1843
Getting next venues for 5: 55691fdc498e0b3c1676862b
Getting next venues for 6: 569f00f938faf971e4fca795
Getting next venues for 7: 5407c3ee498e743eaa28f797
Getting next venues for 8: 553fe7b3498e44acdf45e71d
Getting next venues for 9: 50c8d0f9e4b033901889acc9
Getting next venues for 10: 53a639f2498ea115343b351d
Getting next venues for 11: 4b88a94ef964a5205c0732e3
Getting next venues for 12: 4aa6b24ff964a520b24a20e3
Getting next venues for 13: 4bf313f7370e76b0c956bd4a
Getting next venues for 14: 4a6f9b3af964a52097d61fe3
Getting next venues for 15: 4f8b1e6bbe77b15e3ac57be0
Getting next venues for 16: 48a4083df964a52090511fe3
Getting next venues for 17: 4bd7694d35aad13a97d88ef3
Getting next venues for 18: 54e76132

Getting next venues for 154: 4a390f6cf964a520179f1fe3
Getting next venues for 155: 56195ac6498e228012dfd22d
Getting next venues for 156: 562405f6498e32f69e70a7a3
Getting next venues for 157: 4db22d538154f5a745d9c093
Getting next venues for 158: 4fc7c071e4b06e4ecff8e93d
Getting next venues for 159: 5bb7d7e92bf29b002dfe55f1
Getting next venues for 160: 4baacbb2f964a52021873ae3
Getting next venues for 161: 457ff3d0f964a520553f1fe3
Getting next venues for 162: 47bae8a8f964a520d14d1fe3
Getting next venues for 163: 4a353e30f964a520ab9c1fe3
Getting next venues for 164: 4a05c35ff964a52072721fe3
Getting next venues for 165: 40f71a80f964a520b20a1fe3
Getting next venues for 166: 512831dae4b058a839d3cdac
Getting next venues for 167: 4c3a462d3849c92891b7c2b1
Getting next venues for 168: 4b4a8ae1f964a5204c8a26e3
Getting next venues for 169: 49ee35c8f964a5203b681fe3
Getting next venues for 170: 4a5a9aeff964a520a4ba1fe3
Getting next venues for 171: 49f796fff964a520c06c1fe3
Getting next venues for 172:

Getting next venues for 306: 4a3f0ce4f964a520aca31fe3
Getting next venues for 307: 43470c00f964a52080281fe3
Getting next venues for 308: 3fd66200f964a5205ded1ee3
Getting next venues for 309: 3fd66200f964a52041ee1ee3
Getting next venues for 310: 5344bed4498e03668d194a8c
Getting next venues for 311: 4827b8b1f964a520bb4f1fe3
Getting next venues for 312: 5459e645498e480913b7ee7b
Getting next venues for 313: 513924afe4b066811795768e
Getting next venues for 314: 44646408f964a52026331fe3
Getting next venues for 315: 574254c7498e1eb5e05a9d57
Getting next venues for 316: 4f62a163e4b08b476db9854e
Getting next venues for 317: 4249ec00f964a5208f201fe3
Getting next venues for 318: 522926b5498e3269edeba28d
Getting next venues for 319: 51bcdea3498ef020aac67972
Getting next venues for 320: 44e37004f964a52042371fe3
Getting next venues for 321: 4460d38bf964a5200a331fe3
Getting next venues for 322: 43fb0c0ef964a520df2f1fe3
Getting next venues for 323: 4feddd79d86cd6f22dc171a9
Getting next venues for 324:

Getting next venues for 458: 4d7009c9516b8cfae0615b10
Getting next venues for 459: 58a3a06dbbec6606e1b580a3
Getting next venues for 460: 5b199f280d8a0f002ccbf30b
Getting next venues for 461: 5258a7db11d2c9469e715fd8
Getting next venues for 462: 49c94f32f964a52054581fe3
Getting next venues for 463: 49dcfa66f964a520cf5f1fe3
Getting next venues for 464: 4d35122d8c42a1cd7c9e005e
Getting next venues for 465: 4dbdfe0dcda109aa6cc0f05a
Getting next venues for 466: 511c234119a980d7108a70b9
Getting next venues for 467: 40e4a580f964a520f5091fe3
Getting next venues for 468: 49dd8602f964a52018601fe3
Getting next venues for 469: 4451c80ef964a520a5321fe3
Getting next venues for 470: 4c7c6e777a856dcb395fe2a7
Getting next venues for 471: 55319ec2498ee038ee77c144
Getting next venues for 472: 4f3deef5e4b055e968f69da8
Getting next venues for 473: 4a0b8ce1f964a52000751fe3
Getting next venues for 474: 4c27c151ed0ac9b6ee3a5faa
Getting next venues for 475: 49ca8f4df964a520b9581fe3
Getting next venues for 476:

Getting next venues for 610: 5674ee88498e64d8f28af57d
Getting next venues for 611: 54dfed34498edb2219438f71
Getting next venues for 612: 51e60310498eca96abdd4a22
Getting next venues for 613: 49f5fab6f964a520e26b1fe3
Getting next venues for 614: 5195394d498e344eeb952b4f
Getting next venues for 615: 4c00513e92a6c928d50944e2
Getting next venues for 616: 4b5924aaf964a520c17e28e3
Getting next venues for 617: 4a947010f964a5209b2120e3
Getting next venues for 618: 50a1bb99e4b006efbc783ede
Getting next venues for 619: 51e2ffc9498eb7f2c84a1f04
Getting next venues for 620: 4babef10f964a520d1d63ae3
Getting next venues for 621: 59c0054f603d2a17e378089a
Getting next venues for 622: 4a0f4909f964a52047761fe3
Getting next venues for 623: 4b78bc3ff964a520fddf2ee3
Getting next venues for 624: 4aca8a43f964a5204ac220e3
Getting next venues for 625: 4ec020bbb8f7963bcdde0f6b
Getting next venues for 626: 4c80518374d7b60c92da6cd8
Getting next venues for 627: 4a692042f964a520a5cb1fe3
Getting next venues for 628:

...

Getting next venues for 637: 4be5efc0bcef2d7f5f9004e5
Getting next venues for 638: 5316829e498ea5c394574bfe
Getting next venues for 639: 4405f0aef964a52041301fe3
Getting next venues for 640: 4a5b6f49f964a5202abb1fe3
Getting next venues for 641: 541615c7498e740c658551cc
Getting next venues for 642: 4192ab80f964a520021e1fe3
Getting next venues for 643: 4a2b6e43f964a520ce961fe3
Getting next venues for 644: 4197f180f964a520131e1fe3
Getting next venues for 645: 45362874f964a520a13b1fe3
Getting next venues for 646: 519fcb37498e91a13cb23d6b
Getting next venues for 647: 58facb86e7a2375f7ec4a690
Getting next venues for 648: 4aaeac2af964a520e06220e3
Getting next venues for 649: 49bee717f964a520f3541fe3
Getting next venues for 650: 4cae4729562d224b8fe90488
Getting next venues for 651: 49d28e9ff964a520b85b1fe3
Getting next venues for 652: 4a53a9e0f964a52096b21fe3
Getting next venues for 653: 4c882fe8ed0aa14307a1a7f3
Getting next venues for 654: 5a20cdb2e185f32e3a680721
Getting next venues for 655:

Getting next venues for 789: 49ee0cfef964a52016681fe3
Getting next venues for 790: 4b1c92fbf964a520800824e3
Getting next venues for 791: 4a0dd9c7f964a520a4751fe3
Getting next venues for 792: 4a5a669ff964a52048ba1fe3
Getting next venues for 793: 4b4f2a0ef964a520eafb26e3
Getting next venues for 794: 50b50e8ce4b0eb094c2ac8ae
Getting next venues for 795: 4b899bb1f964a520c04532e3
Getting next venues for 796: 4ad365bcf964a52025e420e3
Getting next venues for 797: 4a8c6c4af964a520f10d20e3
Getting next venues for 798: 5aea242f628c83002c960cae
Getting next venues for 799: 4ae222a1f964a5200f8b21e3
Getting next venues for 800: 46d715faf964a5206c4a1fe3
Getting next venues for 801: 4b771178f964a520fb792ee3
Getting next venues for 802: 4c1c417deac020a16f5a46c2
Getting next venues for 803: 4d436bf014aa8cfa86815a3d
Getting next venues for 804: 4b539f6ff964a5200da527e3
Getting next venues for 805: 4c4ba143c668e21e85bd4dfa
Getting next venues for 806: 5aa14d004186865d6c603888
Getting next venues for 807:

Getting next venues for 941: 4a85b67ef964a52004ff1fe3
Getting next venues for 942: 4bc7977914d79521c4b167e9
Getting next venues for 943: 4e4cc8a5bd413c4cc66c2288
Getting next venues for 944: 4b880cb4f964a520c0db31e3
No. of venues: 294
Getting next venues for 0: 59bb7bd30531e83b964b6cb2
Getting next venues for 1: 4aeb400bf964a52020c021e3
Getting next venues for 2: 4ab90c8bf964a520d27d20e3
Getting next venues for 3: 4ab69f18f964a520347820e3
Getting next venues for 4: 471d30a0f964a520b64b1fe3
Getting next venues for 5: 4b5f84ecf964a5202ebf29e3
Getting next venues for 6: 4a6ab0a4f964a520d9cd1fe3
Getting next venues for 7: 4a593c98f964a52012b91fe3
Getting next venues for 8: 52f6ef60498e618cf9fa1296
Getting next venues for 9: 45301159f964a5205d3b1fe3
Getting next venues for 10: 40870b00f964a520b3f21ee3
Getting next venues for 11: 49c177c4f964a520ad551fe3
Getting next venues for 12: 56351c74498ea61bf071d388
Getting next venues for 13: 53f5289d498edd2444248aa0
Getting next venues for 14: 49e4d

Getting next venues for 150: 4b89cc7cf964a520db4d32e3
Getting next venues for 151: 4bba487d935e952162f62790
Getting next venues for 152: 4b44f01ef964a520460026e3
Getting next venues for 153: 45602bddf964a520963d1fe3
Getting next venues for 154: 5357ee24498efe80c8e2b8d7
Getting next venues for 155: 49e98651f964a520fc651fe3
Getting next venues for 156: 53ab8b36498efd284f85d7b9
Getting next venues for 157: 450dc824f964a5205c391fe3
Getting next venues for 158: 4a03132af964a52087711fe3
Getting next venues for 159: 441c222df964a52051311fe3
Getting next venues for 160: 537aa1d1498eddb91b3586f5
Getting next venues for 161: 4ac912e5f964a52073be20e3
Getting next venues for 162: 4b75d8b7f964a5200a292ee3
Getting next venues for 163: 4b33f540f964a520dc2225e3
Getting next venues for 164: 55fcc2bb498e0c5ea6a4c51f
Getting next venues for 165: 5303f1c9498eb29577d2d465
Getting next venues for 166: 51d25a898bbd024bb44bff91
Getting next venues for 167: 4a6fa61ff964a520b5d61fe3
Getting next venues for 168:

Getting next venues for 8: 51f812d28bbdfc0de9290b1a
Getting next venues for 9: 4b04bb5ef964a520675622e3
Getting next venues for 10: 4bd34ee9caff9521e694d4f0
Getting next venues for 11: 4eff82622c5b66f33bfb1c60
Getting next venues for 12: 4e8525f7f5b9343141e2d64f
Getting next venues for 13: 4bc8a0b08b7c9c74908838cf
Getting next venues for 14: 4abb9a4ef964a520188420e3
Getting next venues for 15: 548a063b498ebda5deae4f7a
Getting next venues for 16: 54448ba1498e71ae5f7a3269
Getting next venues for 17: 4a425ecef964a520cba51fe3
Getting next venues for 18: 5126d718e4b02757cdf6627b
Getting next venues for 19: 4ae11e11f964a520518521e3
Getting next venues for 20: 4c7055c39c6d6dcbab64d27a
Getting next venues for 21: 52c46ddd498ee4522203da6f
Getting next venues for 22: 46a2379ef964a52028491fe3
Getting next venues for 23: 4db47f575da3a76f443fe3fa
Getting next venues for 24: 4da1138663b5a35d7339e119
Getting next venues for 25: 4c539eb5728920a130a96181
Getting next venues for 26: 49d062caf964a520225b

In [494]:
venues = new_venues
venues.drop('gotNextVenues', axis=1, inplace=True)
venues.index.name = 'venue_id'
venues.to_pickle("data/venues.pkl")
venues.shape

(5549, 10)

In [495]:
venues.head()

,address,category,city,country,cross_street,lat,lng,name,postal_code,state
venue_id,,,,,,,,,,
4f3709abe4b0aa62a1d35520,748 Innes Ave,4bf58dd8d48988d1ed941735,San Francisco,United States,NaN,37.730525,-122.372998,Archimedes Banya,94124,CA
49d3e08bf964a5201f5c1fe3,1634 Jerrold Ave,4bf58dd8d48988d11b951735,San Francisco,United States,NaN,37.739589,-122.390171,Flora Grubb Gardens,94124,CA
53058fdb498e2837542f7f8f,1439 Egbert Ave Unit c,4e0e22f5a56208c4ea9a85a0,San Francisco,United States,3rd Street,37.723004,-122.393615,Seven Stills Brewery & Distillery,94124,CA
555e58ad498e9137bc49f191,900 Marin St,4bf58dd8d48988d1e5931735,San Francisco,United States,btwn Illinois & Michigan St,37.749404,-122.386016,The Midway SF,94124,CA
56eb4e0a498e5769545c1843,1525 Cortland Ave,50327c8591d4c4b30a586d5d,San Francisco,United States,NaN,37.739798,-122.408986,Barebottle Brewing Company,NaN,CA
